# Denoising Autoencoder

## Overview

In this notebook, we will use what we covered in [`0-autoencoder.ipynb`](0-autoencoder.ipynb) regarding autoencoder, and use it for denoising.

## Setup
Let's start by importing the libraries and functions that we will need.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

__version__ = '1.0.0'
__author__ = 'Abien Fred Agarap'

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.datasets import mnist

tf.config.experimental.set_memory_growth(tf.config.experimental.list_physical_devices('GPU')[0], True)
tf.random.set_seed(1)
np.random.seed(1)

print('TensorFlow version:', tf.__version__)
print('Is Executing Eagerly?', tf.executing_eagerly())

TensorFlow version: 2.0.0
Is Executing Eagerly? True


## Autoencoder model

An autoencoder consists of two components: (1) an **encoder** which learns the data representation $z$, i.e. the important features of a given data $x$ (I like to describe it as _what makes something something_), and (2) a **decoder** which outputs a reconstruction of the data $\hat{x}$ based on its idea $z$ of how the original data $x$ is structured.
$$ z = f\big(h_{e}(x)\big)$$
$$ \hat{x} = f\big(h_{d}(z)\big)$$
where $z$ is the learned data representation by encoder $h_{e}$, and $\hat{x}$ is the reconstructed data by decoder $h_{d}$ based on $z$.

Let's further dissect the model below.

### Define an encoder layer

The first component, the **encoder**, is similar to a conventional feed-forward network. However, it is not tasked on predicting values (a _regression_ task) or categories (a _classification_ task). Instead, it is tasked to learn how the data is structured, i.e. data representation $z$. We can implement the encoder layer as follows,

In [2]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, intermediate_dim=128, code_dim=64):
        super(Encoder, self).__init__()
        self.hidden_layer = tf.keras.layers.Dense(units=intermediate_dim, activation=tf.nn.relu)
        self.output_layer = tf.keras.layers.Dense(units=code_dim, activation=tf.nn.sigmoid)
    
    def call(self, input_features):
        activation = self.hidden_layer(input_features)
        return self.output_layer(activation)

The _encoding_ is done by passing data input $x$ to the encoder's hidden layer $h$ in order to learn the data representation $z = f(h(x))$.

We first write an `Encoder` class that inherits the [`tf.keras.layers.Layer`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer) class to define it as a layer. So, why a layer instead of a model? Recall that the encoder is a _component_ of the autoencoder model.

Analyzing the code, the `Encoder` layer is defined to have a single hidden layer of neurons (`self.hidden_layer`) to learn the input features. Then, we connect the hidden layer to a layer (`self.output_layer`) that encodes the learned activations to lower dimension which consists of what it thinks as important features. Hence, the "output" of the `Encoder` layer is the _what makes something something_ $z$ of the data $x$.

### Define a decoder layer

The second component, the **decoder**, is also similar to a feed-forward network. However, instead of reducing data to lower dimension, it attempts to reverse the process, i.e. reconstruct the data from its lower dimension representation $z$ to its original dimension.

The _decoding_ is done by passing the lower dimension representation $z$ to the decoder's hidden layer $h$ in order to reconstruct the data to its original dimension $\hat{x} = f(h(z))$. We can implement the decoder layer as follows,

In [3]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, original_dim, code_dim=64):
        super(Decoder, self).__init__()
        self.hidden_layer = tf.keras.layers.Dense(units=code_dim, activation=tf.nn.relu)
        self.output_layer = tf.keras.layers.Dense(units=original_dim, activation=tf.nn.sigmoid)
  
    def call(self, code):
        activation = self.hidden_layer(code)
        return self.output_layer(activation)

We now write a `Decoder` class that also inherits the `tf.keras.layers.Layer`.

The `Decoder` layer is also defined to have a single hidden layer of neurons to reconstruct the input features $\hat{x}$ from the learned representation $z$ by the encoder $f\big(h_{e}(x)\big)$. Then, we connect its hidden layer to a layer that decodes the data representation from lower dimension $z$ to its original dimension $\hat{x}$. Hence, the "output" of the `Decoder` layer is the reconstructed data $\hat{x}$ from the data representation $z$.

Ultimately, the output of the decoder is the autoencoder's output.

Now that we have defined the components of our autoencoder, we can finally build our model.

### Build the autoencoder model

We can now build the autoencoder model by instantiating `Encoder` and `Decoder` layers.

In [4]:
class Autoencoder(tf.keras.Model):
    def __init__(self, intermediate_dim, code_dim, original_dim):
        super(Autoencoder, self).__init__()
        self.loss = []
        self.encoder = Encoder(intermediate_dim=intermediate_dim, code_dim=code_dim)
        self.decoder = Decoder(code_dim=code_dim, original_dim=original_dim)

    def call(self, input_features):
        code = self.encoder(input_features)
        reconstructed = self.decoder(code)
        return reconstructed

As discussed above, the encoder's output is the input to the decoder, as it is written above (`reconstructed = self.decoder(code)`).

### Reconstruction error

We only discussed and built the model, but we talked about how it actually learns. All we know up to this point is the _flow of learning_ from the input layer of the encoder which supposedly learns the data representation, and use that representation as input to the decoder that reconstructs the original data.
Like "simple" neural networks, an autoencoder learns through [backpropagation](https://www.youtube.com/watch?v=LOc_y67AzCA). However, instead of comparing the values or labels of the model, we compare the reconstructed data and the original data. Let's call this comparison the reconstruction error function, and it is given by the following equation,
$$ L = \dfrac{1}{n} \sum_{i=0}^{n-1} \big(\hat{x}_{i} - x_{i}\big)^{2}$$
where $\hat{x}$ is the reconstructed data while $x$ is the original data.

We can implement it as follows,

In [5]:
def loss(reconstructed, original):
    return tf.reduce_mean(tf.square(tf.subtract(reconstructed, original)))

### Forward pass and optimization

We will write a function for computing the forward pass, and applying a chosen optimization function.

In [6]:
def train(loss, model, opt, original):
    with tf.GradientTape() as tape:
        reconstructed = model(original)
        reconstruction_error = loss(reconstructed, original)
    gradients = tape.gradient(reconstruction_error, model.trainable_variables)
    gradient_variables = zip(gradients, model.trainable_variables)
    opt.apply_gradients(gradient_variables)
    return reconstruction_error

### The training loop

Finally, we will write a function to run the training loop. This function will take arguments for the model, the optimization function, the loss, the dataset, and the training epochs.

The training loop itself uses a `GradientTape` context defined in `train` for each batch.

In [7]:
def train_loop(model, opt, loss, dataset, epochs):
    for epoch in range(epochs):
        epoch_loss = 0
        for batch_features in dataset:
            loss_values = train(loss, model, opt, batch_features)
            epoch_loss += loss_values
        model.loss.append(tf.reduce_mean(epoch_loss))
        print('Epoch {}/{}. Loss: {}'.format(epoch + 1, epochs, tf.reduce_mean(epoch_loss)))

### Process the dataset

Now that we have defined our `Autoencoder` class, the loss function, and the training loop, let's import the dataset. We will normalize the pixel values for each example through dividing by maximum pixel value. We shall flatten the examples from 28 by 28 arrays to 784-dimensional vectors.

In [8]:
(training_features, _), (test_features, _) = mnist.load_data()

training_features = training_features.astype('float32') / 255.
test_features = test_features.astype('float32') / 255.

training_features += tf.random.normal(stddev=5e-2, shape=training_features.shape)
test_features += tf.random.normal(stddev=5e-2, shape=test_features.shape)

training_features = training_features.reshape(training_features.shape[0], 784)
test_features = test_features.reshape(test_features.shape[0], 784)

training_dataset = tf.data.Dataset.from_tensor_slices(training_features)
training_dataset = training_dataset.prefetch(4096)
training_dataset = training_dataset.shuffle(training_features.shape[0])
training_dataset = training_dataset.batch(128, True)

## Train the model

Now, all we have to do is instantiate the autoencoder model and choose an optimization function, then pass the intermediate dimension and the original dimension of the images.

In [ ]:
model = Autoencoder(intermediate_dim=128, code_dim=64, original_dim=784)
opt = tf.optimizers.Adam(learning_rate=1e-2)

train_loop(model, opt, loss, training_dataset, 100)

Epoch 1/100. Loss: 24.58465003967285
Epoch 2/100. Loss: 12.778584480285645
Epoch 3/100. Loss: 9.938030242919922
Epoch 4/100. Loss: 8.548629760742188
Epoch 5/100. Loss: 7.863142967224121
Epoch 6/100. Loss: 7.4014997482299805
Epoch 7/100. Loss: 7.0308918952941895
Epoch 8/100. Loss: 6.7640204429626465
Epoch 9/100. Loss: 6.610593795776367
Epoch 10/100. Loss: 6.525899887084961
Epoch 11/100. Loss: 6.405040740966797
Epoch 12/100. Loss: 6.370055198669434
Epoch 13/100. Loss: 6.298134803771973
Epoch 14/100. Loss: 6.250308513641357
Epoch 15/100. Loss: 6.235457897186279
Epoch 16/100. Loss: 6.215330123901367
Epoch 17/100. Loss: 6.11767053604126
Epoch 18/100. Loss: 6.017238616943359
Epoch 19/100. Loss: 5.956178188323975
Epoch 20/100. Loss: 5.871927738189697
Epoch 21/100. Loss: 5.847806930541992
Epoch 22/100. Loss: 5.801619052886963
Epoch 23/100. Loss: 5.776648044586182
Epoch 24/100. Loss: 5.76164436340332
Epoch 25/100. Loss: 5.743459224700928
Epoch 26/100. Loss: 5.722104549407959
Epoch 27/100. Loss:

## Plot the in-training performance

Let's take a look at how the model performed during training in a couple of plots.

In [ ]:
sns.setstyle('dark', {'grid.linestyle': '--'})
plt.plot(range(100), model.loss)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid()
plt.show()

## Predictions

Finally, we will look at some of the predictions. The wrong predictions are labeled in red color.

In [ ]:
number = 10
plt.figure(figsize=(20, 4))
for index in range(number):
    # display original
    ax = plt.subplot(2, number, index + 1)
    plt.imshow(test_features[index].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, number, index + 1 + number)
    plt.imshow(model(test_features)[index].numpy().reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## Closing remarks

As you may see after training this model, the reconstructed images are quite blurry. A number of things could be done to move forward from this point, e.g. adding more layers, or using a convolutional neural network architecture as the basis of the autoencoder, or use a different kind of autoencoder.

## References
* Martín Abadi, Ashish Agarwal, Paul Barham, Eugene Brevdo, Zhifeng Chen, Craig Citro, Greg S. Corrado, Andy Davis, Jeffrey Dean, Matthieu Devin, Sanjay Ghemawat, Ian Goodfellow, Andrew Harp, Geoffrey Irving, Michael Isard, Rafal Jozefowicz, Yangqing Jia, Lukasz Kaiser, Manjunath Kudlur, Josh Levenberg, Dan Mané, Mike Schuster, Rajat Monga, Sherry Moore, Derek Murray, Chris Olah, Jonathon Shlens, Benoit Steiner, Ilya Sutskever, Kunal Talwar, Paul Tucker, Vincent Vanhoucke, Vijay Vasudevan, Fernanda Viégas, Oriol Vinyals, Pete Warden, Martin Wattenberg, Martin Wicke, Yuan Yu, and Xiaoqiang Zheng. TensorFlow: Large-scale machine learning on heterogeneous systems, 2015. Software available from [tensorflow.org](https://tensorflow.org/).
* Chollet, F. (2016, May 14). Building Autoencoders in Keras. Retrieved March 19, 2019, from https://blog.keras.io/building-autoencoders-in-keras.html
* Goodfellow, I., Bengio, Y., & Courville, A. (2016). Deep learning. MIT press.